In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\82106\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# 모델 로드
model = SentenceTransformer("jhgan/ko-sbert-sts")
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
llm_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

c:\Users\82106\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\82106\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [3]:
file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\train.csv"
train = pd.read_csv(file_path)

file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\test.csv"
test = pd.read_csv(file_path)

file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\sample_submission.csv"
sample = pd.read_csv(file_path)

### 프로세스
- 데이터 그룹화 
- -> 반복문을 통해 각 사고 유형별 대표 대책 문장 선정 (코사인 유사도를 통해 유사도(보편도)를 측정) 
- -> RAG 적용을 위한 프롬프트 생성 및 LLM 처리

In [4]:
grouped = train.groupby("인적사고")

res = {}
res_enhanced = {}
cosine_res = []

for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    sentences = plan.tolist()
    vectors = model.encode(sentences, batch_size=32, show_progress_bar=True)

    similarity = cosine_similarity(vectors, vectors)
    best_idx = similarity.mean(axis=1).argmax()

    cosine_res += similarity[best_idx].tolist()
    representative_plan = plan.iloc[best_idx]
    res[name] = representative_plan

    # RAG 적용 - 대회 규칙 준수를 위한...
    rag_prompt = f"""
    너가 받은 내용 그대로만 말하면 된다.
    너가 멋대로 뭐 바꾸거나 추가하면 안된다. 그냥 받은 내용 그대로 말하면 된다. 명심해.

    {representative_plan}"""

    messages = [{"role": "system", "content": "전달받은 내용을 단 한 글자도 바꾸지 않고 완전히 그대로 출력합니다."}, {"role": "user", "content": rag_prompt}]

    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

    output = llm_model.generate(
        input_ids.to(llm_model.device),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=256,
        do_sample=False,
        temperature=None,
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    assistant_response = generated_text.split("[|assistant|]")[-1].strip()
    res_enhanced[name] = assistant_response

  0%|          | 0/23 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
res_v = {}
res_enhanced_v = {}

for k, v in res.items():
    res_v[k] = model.encode(v)
    res_enhanced_v[k] = model.encode(res_enhanced[k])

for i in range(len(test)):
    accident = test.loc[i, "인적사고"]

    if accident in res_enhanced:
        sample.loc[i, "재발방지대책 및 향후조치계획"] = res_enhanced[accident]
        sample.iloc[i, 2:] = res_enhanced_v[accident]

In [6]:
sample.to_csv("C:/Users/82106/Desktop/데이콘 한솔데크/data/result/baseline+RAG.csv", index=False, encoding='utf-8-sig')